In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
#recuperamos la salida del proceso04
departamentos = pd.read_csv('../data/out_proceso04.csv', decimal=".")

departamentos.fillna(0, inplace=True)

departamentos.columns

In [ ]:
#eliminamos variables que no vamos a usar en el modelo
departamentos.drop(['operacion', 'tipo_propiedad', 'pais', 'provincia', 'ciudad_barrio','emprendimiento',
                    'barrio', 'barrio_cerrado', 'geonames_url', 'precio', 'moneda', 'titulo', 'descripcion',
                    'descripciontrip','geometry'], axis=1, inplace=True)

In [ ]:
departamentos.columns

In [ ]:
departamentos_sample = departamentos.sample(frac=1.0, random_state = 42)

X = departamentos_sample.drop('valor_m2', axis=1)

y = departamentos_sample[['valor_m2']]

## Sin PolynomialFeatures

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
se = StandardScaler()
X_train = se.fit_transform(X_train)
X_test = se.fit_transform(X_test)

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 35, 40, 50, 60]

alpha_ridge =[]
mean_ridge = []
std_ridge =[]
predict_ridge = []

def f_ridge(alpha, X_train, X_test, y_train, y_test):
    model_ridge = Ridge(alpha=alpha, normalize=False)
    results_ridge = cross_val_score(model_ridge, X_train, y_train, cv=5, scoring='r2')
    alpha_ridge.append(alpha)
    mean_ridge.append(np.mean(results_ridge))
    std_ridge.append(np.std(results_ridge))
    model_ridge.fit(X_train,y_train)
    predict_ridge.append(model_ridge.predict(X_test))

for alpha in alphas:
    f_ridge(alpha, X_train, X_test, y_train, y_test)
    

alpha_lasso =[]
mean_lasso = []
std_lasso =[]
predict_lasso = []

def f_lasso(alpha, X_train, X_test, y_train, y_test):
    model_lasso = Lasso(alpha=alpha, max_iter=10000, normalize=False) 
    results_lasso = cross_val_score(model_lasso, X_train, y_train, cv=5, scoring='r2')
    alpha_lasso.append(alpha)
    mean_lasso.append(np.mean(results_lasso))
    std_lasso.append(np.std(results_lasso))
    model_lasso.fit(X_train,y_train)
    predict_lasso.append(model_lasso.predict(X_test))

for alpha in alphas:
    f_lasso(alpha, X_train, X_test, y_train, y_test)

plt.plot(alpha_lasso, mean_lasso, label='lasso')
plt.plot(alpha_ridge, mean_ridge, label='Ridge')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot the model - predict_ridge - alpha 35

predict_ridge_list = []
for predict in  predict_ridge[27]:
    predict_ridge_list.append(predict)

#for predict in predict_lasso:
dfPredict_ridge = pd.DataFrame({'predict_ridge':predict_ridge_list})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_ridge, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()
print(r2_score(y_test, dfPredict_ridge))
print ("MSE:", mean_squared_error(y_test, dfPredict_ridge))

In [ ]:
# Plot the model - predict_lasso - alpha 7

#for predict in predict_lasso:
dfPredict_lasso = pd.DataFrame({'predict_lasso':predict_lasso[11]})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_lasso, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()

print(r2_score(y_test, dfPredict_lasso))
print ("MSE:", mean_squared_error(y_test, dfPredict_lasso))

## Con PolynomialFeatures

In [ ]:
p = PolynomialFeatures(degree=2).fit(X)
X_features = pd.DataFrame(p.transform(X), columns=p.get_feature_names(X.columns))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=53)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

se = StandardScaler()
X_train = se.fit_transform(X_train)
X_test = se.fit_transform(X_test)

In [ ]:
alpha_ridge =[]
mean_ridge = []
std_ridge =[]
predict_ridge = []

def f_ridge(alpha, X_train, X_test, y_train, y_test):
    model_ridge = Ridge(alpha=alpha, normalize=False)
    results_ridge = cross_val_score(model_ridge, X_train, y_train, cv=5, scoring='r2')
    alpha_ridge.append(alpha)
    mean_ridge.append(np.mean(results_ridge))
    std_ridge.append(np.std(results_ridge))
    model_ridge.fit(X_train,y_train)
    predict_ridge.append(model_ridge.predict(X_test))

alphas = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]

for alpha in alphas:
    f_ridge(alpha, X_train, X_test, y_train, y_test)
    

alpha_lasso =[]
mean_lasso = []
std_lasso =[]
predict_lasso = []

def f_lasso(alpha, X_train, X_test, y_train, y_test):
    model_lasso = Lasso(alpha=alpha, max_iter=10000, normalize=False) 
    results_lasso = cross_val_score(model_lasso, X_train, y_train, cv=5, scoring='r2')
    alpha_lasso.append(alpha)
    mean_lasso.append(np.mean(results_lasso))
    std_lasso.append(np.std(results_lasso))
    model_lasso.fit(X_train,y_train)
    predict_lasso.append(model_lasso.predict(X_test))

for alpha in alphas:
    f_lasso(alpha, X_train, X_test, y_train, y_test)

plt.plot(alpha_lasso, mean_lasso, label='lasso')
plt.plot(alpha_ridge, mean_ridge, label='Ridge')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
alpha_ridge =[]
mean_ridge = []
std_ridge =[]
predict_ridge = []

def f_ridge(alpha, X_train, X_test, y_train, y_test):
    model_ridge = Ridge(alpha=alpha, normalize=False)
    results_ridge = cross_val_score(model_ridge, X_train, y_train, cv=5, scoring='r2')
    alpha_ridge.append(alpha)
    mean_ridge.append(np.mean(results_ridge))
    std_ridge.append(np.std(results_ridge))
    model_ridge.fit(X_train,y_train)
    predict_ridge.append(model_ridge.predict(X_test))

alphas =[600, 700, 750, 800]

for alpha in alphas:
    f_ridge(alpha, X_train, X_test, y_train, y_test)
    
plt.plot(alpha_ridge, mean_ridge, label='Ridge')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot the model - predict_ridge - alpha 750

predict_ridge_list = []
for predict in  predict_ridge[2]:
    predict_ridge_list.append(predict)

#for predict in predict_lasso:
dfPredict_ridge = pd.DataFrame({'predict_ridge':predict_ridge_list})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_ridge, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()
print(r2_score(y_test, dfPredict_ridge))
print ("MSE:", mean_squared_error(y_test, dfPredict_ridge))

In [ ]:
alpha_lasso =[]
mean_lasso = []
std_lasso =[]
predict_lasso = []

def f_lasso(alpha, X_train, X_test, y_train, y_test):
    model_lasso = Lasso(alpha=alpha, max_iter=10000, normalize=False) 
    results_lasso = cross_val_score(model_lasso, X_train, y_train, cv=5, scoring='r2')
    alpha_lasso.append(alpha)
    mean_lasso.append(np.mean(results_lasso))
    std_lasso.append(np.std(results_lasso))
    model_lasso.fit(X_train,y_train)
    predict_lasso.append(model_lasso.predict(X_test))

alphas = [30, 30.5, 31, 31.5, 32]
    
for alpha in alphas:
    f_lasso(alpha, X_train, X_test, y_train, y_test)


plt.plot(alpha_lasso, mean_lasso, label='Lasso')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot the model - predict_lasso - alpha 31

#for predict in predict_lasso:
dfPredict_lasso = pd.DataFrame({'predict_lasso':predict_lasso[2]})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_lasso, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()

print(r2_score(y_test, dfPredict_lasso))
print ("MSE:", mean_squared_error(y_test, dfPredict_lasso))